# 常见问题解答

本文档将解决在文档的其他页面中未解决的常见问题。

## 如何安装cobrapy?

请参阅文件 [INSTALL.rst](https://github.com/opencobra/cobrapy/blob/stable/INSTALL.rst) 

##如何引用cobrapy?

请引用2013年的出版物: [10.1186/1752-0509-7-74](http://dx.doi.org/doi:10.1186/1752-0509-7-74)

## 如何重新命名反应或代谢物?

TL;DR 之后使用“Model.repair”

当重命名代谢物或反应时，会出现一些问题，因为cobra索引是基于ID的，这可能会导致错误。例如:

In [1]:
from cobra.io import load_model
model = load_model("iYS1720")

for metabolite in model.metabolites:
    metabolite.id = f"test_{metabolite.id}"

try:
    model.metabolites.get_by_id(model.metabolites[0].id)
except KeyError as e:
    print(repr(e))

Model.repair 函数将重新生成必要的索引

In [2]:
model.repair()
model.metabolites.get_by_id(model.metabolites[0].id)

Metabolite identifier,test_10fthf_c
Name,10-Formyltetrahydrofolate
Memory address,0x16e9f8c5750
Formula,C20H21N7O7
Compartment,c
In 9 reaction(s),"GARFT, MTHFC, BIOMASS_Ec_iAF1260_core_59p81M, BIOMASS_invivo, TDPFRMT, FTHFD, ULA4NFT, AICART, FMETTRS"


## 如何删除基因？

这取决于你所说的删除基因到底是什么意思。

如果要模拟基因敲除的模型，请在上下文中使用`cobra.manipulation.knock_out_model_genes` 函数。退出上下文时，此函数的效果将相反。

In [3]:
model = load_model("iYS1720")
PGI = model.reactions.get_by_id("PGI")
print("bounds before knockout:", (PGI.lower_bound, PGI.upper_bound))
from cobra.manipulation import knock_out_model_genes
knock_out_model_genes(model, ["STM4221"])
print("bounds after knockouts", (PGI.lower_bound, PGI.upper_bound))

bounds before knockout: (-1000.0, 1000.0)
bounds after knockouts (0, 0)


如果你想从模型中去除一个基因的所有痕迹，这要困难得多，因为这需要改变涉及该基因的反应的所有 `gene_reaction_rule` 字符串。

## 如何更改反应的可逆性？

`Reaction.reversibility` 是cobra中的一个属性，在从下界和上界请求时计算。

In [4]:
model = load_model("iYS1720")
model.reactions.get_by_id("PGI").reversibility

True

尝试直接设置它将导致错误或警告：

In [5]:
try:
    model.reactions.get_by_id("PGI").reversibility = False
except Exception as e:
    print(repr(e))

d:\ProgramData\anaconda3\Lib\site-packages\cobra\core\reaction.py:810: UserWarning: Setting reaction reversibility is ignored
  warn("Setting reaction reversibility is ignored")


改变可逆性的方法是改变边界，使反应不可逆。

In [6]:
model.reactions.get_by_id("PGI").lower_bound = 10
model.reactions.get_by_id("PGI").reversibility

False

## 如何从 COBRA 模型生成 LP 文件？

### 关于基于 optlang 的求解器

使用 optlang 求解器，模型的 LP 公式通过其字符串表示获得。所有求解器的行为方式都相同。

In [7]:
with open('test.lp', 'w') as out:
    out.write(str(model.solver))

### 我如何将通量解形象化?

请浏览我们网站上的 [visualization packages](https://opencobra.github.io/cobrapy/tags/visualization/)以获取最新的工具列表。